## From CSV to Mysql to Cassandra

in this Notebook I'll be using Pyspark to: 

1 - read from the tweets csv file into a pyspark dataframe

2 - saving the pyspark dataframe into a Mysql table (raw data before map reduce)

3 - using pyspark to read from mysql table

4 - apply reduce and data transformation to the dataframe

5 - apply data cleaning,data engineering and sentiment analysis 

6 - using pyspark for Saving resulting dataframe (post map-reduce) into Cassandra

7 - reading from cassandra and create a csv as output for continuing with time serie analysis on another notebook


#### connectors for Mysql and Cassandra
pyspark --jars mysql-connector-j-8.1.0.jar --packages com.datastax.spark:spark-cassandra-connector_2.12:3.1.0

In [1]:
#create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .appName("MyApp") \
  .config("spark.jars",  "mysql-connector-j-8.1.0.jar") \
  .master("local")\
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
warnings.filterwarnings("ignore")

Data can be uploaded normally from csv to mysql and then read it with pyspark

in this case I'm uploading csv to mysql using pyspark and then reading it from mysql table to keep all in one notebook

In [2]:
#pyspark read from csv
data = spark.read.csv("/user1/ProjectTweets.csv", inferSchema=True)
data.show()

+---+----------+--------------------+--------+---------------+--------------------+
|_c0|       _c1|                 _c2|     _c3|            _c4|                 _c5|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|  5|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|  6|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|  7|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|  8|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood|@Tatiana_K nop

In [3]:
#pyspark write into Mysql table
# sql database is called Tweets and table is called Tweets, schema is already present in mysql (done through CLI)
data.write \
  .format("jdbc") \
  .mode("overwrite") \
  .option("url", "jdbc:mysql://localhost:3306/Tweets") \
  .option("dbtable", "Tweets") \
  .option("user", "root") \
  .option("password", "password") \
  .save()

In [4]:
#pyspark read from Mysql table we just inserted 
df = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/Tweets") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "Tweets") \
    .option("user", "root").option("password", "password").load()

df.show()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


+---+----------+--------------------+--------+---------------+--------------------+
|_c0|       _c1|                 _c2|     _c3|            _c4|                 _c5|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|  5|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|  6|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|  7|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|  8|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood|@Tatiana_K nop

#### Data engineering using pyspark

In [5]:
if df.count() > 1000000:
    print(f"{data.count()} ... That's a lot of Data!!")

1600000 ... That's a lot of Data!!


In [6]:
#check unique values for _c3
df.select('_c3').distinct().collect()
#field _c3 only has 1 value, dropping field

[Row(_c3='NO_QUERY')]

In [7]:
df = df.drop(df._c3)

In [8]:
#checking for duplicates
df.groupby("_c1").count().where("count > 1").show()

+----------+-----+
|       _c1|count|
+----------+-----+
|1468544973|    2|
|1690908358|    2|
|1834777946|    2|
|1882160717|    2|
|1965601765|    2|
|1982434182|    2|
|2002309001|    2|
|2190980212|    2|
|1685304801|    2|
|1686371908|    2|
|1957194329|    2|
|1969964899|    2|
|1974268607|    2|
|2056807406|    2|
|2063670799|    2|
|1556266702|    2|
|1752414405|    2|
|1824843992|    2|
|1881996107|    2|
|1983726537|    2|
+----------+-----+
only showing top 20 rows



In [9]:
duplicates = df.groupby("_c1").count().where("count > 1").drop("count")
print(f"Number of duplicates: {duplicates.count()}")

Number of duplicates: 1685


In [10]:
#show 1 duplicate example
df[df["_c1"] == 1983726537].show(truncate=False)

+-------+----------+----------------------------+-------+---------------------------------------------------------------------------------------------+
|_c0    |_c1       |_c2                         |_c4    |_c5                                                                                          |
+-------+----------+----------------------------+-------+---------------------------------------------------------------------------------------------+
|252393 |1983726537|Sun May 31 13:42:57 PDT 2009|iargent|Should have gone on a bike ride today but never quite happened  Still enjoyed the sun though |
|1190503|1983726537|Sun May 31 13:42:57 PDT 2009|iargent|Should have gone on a bike ride today but never quite happened  Still enjoyed the sun though |
+-------+----------+----------------------------+-------+---------------------------------------------------------------------------------------------+



In [11]:
df = df.dropDuplicates(['_c1'])
df.count() #checking how many values after dropping duplicates

1598315

In [12]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: long (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)



Dealing with timestamps

In [13]:
#example: 
df.first()["_c2"] #PDT stands for Pacific time zone

'Mon Apr 06 22:32:38 PDT 2009'

In [14]:
#let's check if all time stamps are in PDT
#if all strings have PDT in the timestamp this list should return empty
[x for x in df.rdd.toLocalIterator() if "PDT" not in x['_c2']]

[]

In [15]:
#all timestamps are PDT
from pyspark.sql.functions import to_timestamp
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY") #Had to set as Legacy cause of error 
#spark.conf.set("spark.sql.legacy.timeParserPolicy","CORRECTED") #for return to standard timeparser policy

Time_Format = "E MMM d HH:mm:ss z yyyy"
df = df.withColumn("Timestamp", to_timestamp(df["_c2"], Time_Format))
df = df.drop(df._c2)
df = df.drop(df._c0)
df.show(5)

+----------+-------------+--------------------+-------------------+
|       _c1|          _c4|                 _c5|          Timestamp|
+----------+-------------+--------------------+-------------------+
|1467860144|     Jana1976|@JonathanRKnight ...|2009-04-07 06:32:38|
|1467862225|        hdm42|@vjl also, your w...|2009-04-07 06:33:11|
|1467889791|jennhelvering|Just called Hills...|2009-04-07 06:40:33|
|1467898027|   twitrbug81|@JonathanRKnight ...|2009-04-07 06:42:49|
|1467904302|bsbnumber1fan|@nick_carter Aww ...|2009-04-07 06:44:34|
+----------+-------------+--------------------+-------------------+
only showing top 5 rows



In [16]:
#cleaning and sentiment analysis
import nltk
import re
from pyspark.sql import functions as f
from nltk.stem import PorterStemmer
nltk.download('punkt')  # Download the necessary data for NLTK
nltk.download('stopwords')
stemmer = PorterStemmer()
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from nltk.sentiment import SentimentIntensityAnalyzer
from pyspark.sql.types import StringType
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

def stem(text):
    #remove tags
    text = re.sub(r'#\w+', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    #remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    ##stop words #stemming
    tokens = nltk.word_tokenize(text)
    stemmed_text = ' '.join([stemmer.stem(token) for token in tokens if token not in stop_words ])
    
    return stemmed_text

stem_udf = udf(stem, StringType())
df = df.withColumn("text", stem_udf(df["_c5"]))

[nltk_data] Downloading package punkt to /home/hduser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hduser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/hduser/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [17]:
df.show()

+----------+--------------+--------------------+-------------------+--------------------+
|       _c1|           _c4|                 _c5|          Timestamp|                text|
+----------+--------------+--------------------+-------------------+--------------------+
|1467860144|      Jana1976|@JonathanRKnight ...|2009-04-07 06:32:38|i hate limit lett...|
|1467862225|         hdm42|@vjl also, your w...|2009-04-07 06:33:11|also websit seem ...|
|1467889791| jennhelvering|Just called Hills...|2009-04-07 06:40:33|just call hillson...|
|1467898027|    twitrbug81|@JonathanRKnight ...|2009-04-07 06:42:49|         thought mac|
|1467904302| bsbnumber1fan|@nick_carter Aww ...|2009-04-07 06:44:34|aww nick i like h...|
|1467928749|      calliott|is tireddddddd. w...|2009-04-07 06:51:26|tireddddddd want ...|
|1467946810|TheDarrenxshow|@ilovepie mines t...|2009-04-07 06:56:37|mine im find well...|
|1467968979|     atothebed|@clarianne APRIL ...|2009-04-07 07:02:45|april 9th isnt co...|
|146798738

In [18]:
#drop useless
df = df.drop(df._c5)
df = df.drop(df._c4)

In [19]:
#renaming
df = df.withColumnRenamed("_c1","id")
df = df.withColumnRenamed("Timestamp","timestamp")
df = df.withColumnRenamed("Text","text")

In [20]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- text: string (nullable = true)



In [21]:
#Sentiment extraction

def VAder(x):
    return sia.polarity_scores(x)["compound"]

# Register the VADER function as a UDF (User-Defined Function)
vader_udf = udf(VAder, DoubleType())

# Add the 'sentiment' column to the DataFrame using the UDF
df = df.withColumn("sentiment", vader_udf(df["text"]))

df.printSchema()

root
 |-- id: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- text: string (nullable = true)
 |-- sentiment: double (nullable = true)



In [22]:
from pyspark.sql.functions import to_date, mean
from pyspark.sql import SparkSession

# Group by day and calculate the mean score
result_df = df.groupBy(to_date("timestamp").alias("Day")).agg(mean("sentiment").alias("score"))
result_df = result_df.sort("Day")

In [23]:
result_df.createOrReplaceTempView("aggregate")
res = spark.sql("SELECT min(Day) from aggregate")
res = spark.sql("SELECT max(Day) from aggregate")
res.show()

+----------+
|  max(Day)|
+----------+
|2009-06-25|
+----------+



In [24]:
#create dataframe with all dates within min and max
import pandas as pd

#creating a new Df with same start and end dates
dates = []
for date in pd.date_range(start="2009-04-07",end="2009-06-25"):
    dates.append(date.strftime("%Y-%m-%d"))
    
date_df = spark.createDataFrame(dates,"string").toDF("Day")

In [25]:
result_df.printSchema()

root
 |-- Day: date (nullable = true)
 |-- score: double (nullable = true)



In [26]:
date_df.printSchema()

root
 |-- Day: string (nullable = true)



In [27]:
#left join to have one dataset with nulls also
output = date_df.join(result_df,on='Day',how='left').sort("Day")

In [28]:
output.show(5)

+----------+-------------------+
|       Day|              score|
+----------+-------------------+
|2009-04-07|0.13217767281887322|
|2009-04-08|               null|
|2009-04-09|               null|
|2009-04-10|               null|
|2009-04-11|               null|
+----------+-------------------+
only showing top 5 rows



In [29]:
#matching name columns in cassandra
output = output.withColumnRenamed("Day","day")

In [32]:
#session for Cassandra
spark = SparkSession.builder\
  .appName("MyApp") \
  .master("local[*]")\
  .getOrCreate()

#write into Cassandra
#Cassandra keyspace and table is already created (done through CLI)
output.write\
  .format('org.apache.spark.sql.cassandra')\
  .mode('overwrite')\
  .options(table='tweets_final',keyspace='tweets_final')\
  .option("confirm.truncate", "true")\
  .save()

In [33]:
#read from cassandra
last = spark.read\
  .format('org.apache.spark.sql.cassandra')\
  .options(table='tweets_final',keyspace='tweets_final')\
  .load()
last = last.sort("Day")
last.show(1)

+-------------------+--------------------+
|                day|               score|
+-------------------+--------------------+
|2009-04-07 00:00:00|0.132177672818873220|
|2009-04-08 00:00:00|                null|
+-------------------+--------------------+
only showing top 2 rows



In [35]:
#result csv file is then moved from virtual machine to my personal machine for modelling part
last.write.csv("Downloads/final_output.csv",header=True)